<a href="https://colab.research.google.com/github/Sushil-Acharya/Sushil/blob/main/Fruit_multiple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
# Import some required libraries. Some libraries will be imported later if needed.
import numpy as np 
import pandas as pd 
import keras 
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
import glob
import os

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dense,Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from keras.optimizers import Adamax
from tensorflow import keras
from tensorflow.keras import layers

In [25]:
from sklearn.datasets import load_files
training_directory = "/content/drive/My Drive/Colab Notebooks/Ours/"

def load_images(image_path):
    data = load_files(image_path)
    files = np.array(data['filenames'])
    targets = np.array(data['target'])
    target_labels = np.array(data['target_names'])
    return files,targets,target_labels
    
x_train, y_train,target_labels = load_images(training_directory)


print('Successfully loaded !')
print('Number of training images : ' , x_train.shape[0])


Successfully loaded !
Number of training images :  396


In [26]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.5, stratify=y_train)

In [27]:
no_of_classes = len(np.unique(y_train))
no_of_classes

19

In [28]:
from keras.utils import np_utils
y_train = np_utils.to_categorical(y_train,no_of_classes)
y_test= np_utils.to_categorical(y_test,no_of_classes)
# x_train = np_utils.to_categorical(x_train,no_of_classes)
# x_test= np_utils.to_categorical(x_test,no_of_classes)

In [29]:
from keras.preprocessing.image import array_to_img, img_to_array, load_img

def convert_image_to_array(files):
    images_as_array=[]
    for file in files:
        # Convert to Numpy Array
        images_as_array.append(img_to_array(load_img(file)))
    return images_as_array

x_train = np.array(convert_image_to_array(x_train))
print('Shape of Training set: ',x_train.shape)

x_test = np.array(convert_image_to_array(x_test))
print('Shape of Test set : ',x_test.shape)

print('Shape of one training image ',x_train[0].shape)

Shape of Training set:  (198, 100, 100, 3)
Shape of Test set :  (198, 100, 100, 3)
Shape of one training image  (100, 100, 3)


In [30]:
x_train = x_train.astype('float32')/255
x_test = x_test.astype('float32')/255

# New section

In [8]:
!pip install keras-tuner

     |████████████████████████████████| 71kB 5.1MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp37-none-any.whl size=78938 sha256=9900bf586b398c879dd019a6885c9cecbfb286948b04542e57a01880da2467bd
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp37-none-any.whl size=15356 sha256=aad129387cd045a91370f4d7b80e4519f4c68cf0496e44087b6e015f0e2c6d3e
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [31]:
import numpy as np
import tensorflow as tf
from kerastuner.tuners import RandomSearch

In [32]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=16, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [2,5]),
        activation='relu',
        input_shape=(100,100,3)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [2,5]),
        activation='relu'
    ),
    # keras.layers.Conv2D(
    #     filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
    #     kernel_size=hp.Choice('conv_2_kernel', values = [2,5]),
    #     activation='relu'
    # ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(19, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adamax(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

# def build_model(hp):  # random search passes this hyperparameter() object 
#     model = keras.models.Sequential()
    
#     model.add(Conv2D(hp.Int('input_units',
#                                 min_value=16,
#                                 max_value=64,
#                                 step=16), (3, 3), input_shape=x_train.shape[1:]))
    
#     # model.add(Activation('relu'))
#     # model.add(BatchNormalization())
#     # model.add(MaxPooling2D(pool_size=(2, 2)))

#     for i in range(hp.Int('n_layers', 1, 3)):  # adding variation of layers.
#         model.add(Conv2D(hp.Int(f'conv_{i}_units',
#                                 min_value=16,
#                                 max_value=64,
#                                 step=16), (3, 3)))
#         model.add(Activation('relu'))

#     model.add(Flatten()) 
#     # model.add(Dense(512,activation = 'relu'))
#     # model.add(Dropout(0.5))
#     model.add(Dense(128,activation = 'relu'))
#     model.add(Dropout(0.5))
#     model.add(Dense(19,activation = 'softmax'))


#     model.compile(optimizer=keras.optimizers.Adamax(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
#                   loss="categorical_crossentropy",
#                   metrics=["accuracy"])
    
#     return model

In [33]:
from kerastuner.tuners import RandomSearch

# tuner = RandomSearch(
#     build_model,
#     objective='val_accuracy',
#     max_trials=10,
#     directory='/content/drive/My Drive/Colab Notebooks',
#     project_name='tuner')

tuner = RandomSearch(build_model, objective='val_accuracy',
                     max_trials=10,
                     executions_per_trial=2,
                     directory='/content/drive/My Drive/Colab Notebooks',
                     project_name='tuner1')

# print summary of the search space
tuner.search_space_summary()

Search space summary
Default search space size: 6
conv_1_filter (Int)
{'default': None, 'conditions': [], 'min_value': 16, 'max_value': 128, 'step': 16, 'sampling': None}
conv_1_kernel (Choice)
{'default': 2, 'conditions': [], 'values': [2, 5], 'ordered': True}
conv_2_filter (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 64, 'step': 16, 'sampling': None}
conv_2_kernel (Choice)
{'default': 2, 'conditions': [], 'values': [2, 5], 'ordered': True}
dense_1_units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 128, 'step': 16, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001], 'ordered': True}


In [34]:
tf.__version__

'2.4.1'

In [35]:
tuner.search(x_train, y_train, validation_data=(x_test, y_test), epochs=10)

Trial 10 Complete [00h 02m 07s]
val_accuracy: 0.8055555522441864

Best val_accuracy So Far: 0.8510101139545441
Total elapsed time: 00h 58m 59s
INFO:tensorflow:Oracle triggered exit


In [36]:
tuner.results_summary()

Results summary
Results in /content/drive/My Drive/Colab Notebooks/tuner1
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
conv_1_filter: 128
conv_1_kernel: 5
conv_2_filter: 32
conv_2_kernel: 2
dense_1_units: 48
learning_rate: 0.001
Score: 0.8510101139545441
Trial summary
Hyperparameters:
conv_1_filter: 32
conv_1_kernel: 2
conv_2_filter: 48
conv_2_kernel: 5
dense_1_units: 112
learning_rate: 0.001
Score: 0.8484848439693451
Trial summary
Hyperparameters:
conv_1_filter: 32
conv_1_kernel: 5
conv_2_filter: 48
conv_2_kernel: 2
dense_1_units: 96
learning_rate: 0.001
Score: 0.8409090936183929
Trial summary
Hyperparameters:
conv_1_filter: 32
conv_1_kernel: 5
conv_2_filter: 64
conv_2_kernel: 5
dense_1_units: 112
learning_rate: 0.001
Score: 0.8383838534355164
Trial summary
Hyperparameters:
conv_1_filter: 16
conv_1_kernel: 2
conv_2_filter: 64
conv_2_kernel: 2
dense_1_units: 64
learning_rate: 0.01
Score: 0.8055555522441864
Trial summary
Hyperpara

In [37]:
models = tuner.get_best_models(num_models=2)
models[0].summary()
models[1].summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 96, 96, 128)       9728      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 95, 95, 32)        16416     
_________________________________________________________________
flatten (Flatten)            (None, 288800)            0         
_________________________________________________________________
dense (Dense)                (None, 48)                13862448  
_________________________________________________________________
dense_1 (Dense)              (None, 19)                931       
Total params: 13,889,523
Trainable params: 13,889,523
Non-trainable params: 0
_________________________________________________________________
Model: "sequential"
_________________________________________________________________
Layer (type)            